<a href="https://colab.research.google.com/github/PabloParadaSouto/Automatica/blob/master/Aprendizaje_profundo_robotica_adaptativa_datos_robot_3_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En primer lugar deberéis subir los archivos de datos (lecturas.npy y velocidades.npy) a vuetra cuenta de drive, colgarlos directamente del directorio raíz para que os funcione el código más abajo. Los dos primeros bloques de código solo ejecutarlos una única vez


In [ ]:
from google.colab import drive


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np

drive.mount('/content/drive')
input ("montado, pulsa para seguir y cargar los datos ")



Cargamos los datos, en principio esto sería suficiente con hacerlo una única vez....Se trata de datos grabados del Turtlebot (robot real) mientras seguía una pared, dado que la velocidad lineal es constante vamos a crear un modelo que aprenda a decidir la velocidad angular del robot en funcioń de lo que se percibe desde el láser.

In [ ]:
print ("voy a cargar datos sensores ")
sensores=np.load('/content/drive/MyDrive/lecturas.npy')
print ("cargado ...., ahora voy con velocidades")
velocidades=np.load('/content/drive/MyDrive/velocidades.npy')
print ("hecho ")

voy a cargar datos sensores 
cargado ...., ahora voy con velocidades
hecho 


Ahora normalizamos los datos, revisa como como lo hago, en principio esta celda solo se debería ejecutar una vez


In [ ]:
#En primer lugar hacemos una inspeccion visual de dimensiones para comprobar que todo correcto
print ("dimension de velocidades ",velocidades.shape," y dimension ",velocidades.ndim)
print ("dimension de lecturas ",sensores.shape," y dimension ",sensores.ndim)


sensores=sensores.astype('float32')
velocidades=velocidades.astype('float32')

#Invertimos los sensores para que tenga mas importancia lo que esta cerca
#hacemos luego la normalizacion habitual
sensores=np.reciprocal(sensores)
for i in range(sensores.shape[1]):
    sensores[:,i]=(sensores[:,i]-np.mean(sensores[:,i]))/np.std(sensores[:,i])
    print ("se ha normalizado ",i," el nuevo valor medio ",np.mean(sensores[:,i])," desviacion ",np.std(sensores[:,i]))


velocidades_maximas=np.amax(velocidades, axis=0)
velocidades_minimas=np.amin(velocidades, axis=0)
print ("velocidades_maximas ", velocidades_maximas)
print ("velocidades minimas ",velocidades_minimas)

maximav=np.amax(velocidades[:,1])
minimav=np.amin(velocidades[:,1])

velocidades[:,1]=(velocidades[:,1]-minimav)/(maximav-minimav)
print ("se ha normalizado  angular el nuevo valor medio ",np.mean(velocidades[:,1])," desviacion ",np.std(velocidades[:,1]))
print  ("Ya he finalizado toda la normalizacion ")


Construimos los conjuntos de entrenamiento y test....

In [ ]:
cardinal_entrena=(int)(round(sensores.shape[0]*(3/4)))
print  ("numero de patrones de entrenamiento ",cardinal_entrena)
x_train=sensores[0:cardinal_entrena,:]
y_train=velocidades[0:cardinal_entrena,1]
y_train=y_train.reshape(y_train.shape[0],1)


x_test=sensores[cardinal_entrena:sensores.shape[0],:]
y_test=velocidades[cardinal_entrena:velocidades.shape[0],1]
y_test=y_test.reshape(y_test.shape[0],1)

print ("shape de entradas entrenamiento ",x_train.shape)
print ("shape de salidas entrenamiento ",y_train.shape)
print ("shape de entradas test ",x_test.shape)
print ("shape de salidas test ",y_test.shape)

numero de patrones de entrenamiento  10070
shape de entradas entrenamiento  (10070, 1438)
shape de salidas entrenamiento  (10070, 1)
shape de entradas test  (3357, 1438)
shape de salidas test  (3357, 1)


Ahora vamos a crear una red profunda, es importante que te fijes en varias cosas:

1. En primer lugar fíjate que tenemos que introducir tensores (numero de dimensiones=3)

x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)

y_train=y_train.reshape(y_train.shape[0],y_train.shape[1],1)

print ("shape x_train ",x_train.shape,"shape y_train ",y_train.shape)

Ahora vamos a crear topologías imposibles!!!!, mira si eres capaz de entender la concatenación de redes, el profesor os lo expliará en clase



In [ ]:
#definimos valores importantes
epocas=200
tamanho_batch=100


capa_oculta_1=20
capa_oculta_2=20
capa_oculta_3=20
#verbose indica lo que se ve durante el entrenamiento
#By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.
#verbose=0 will show you nothing (silent)
#verbose=1 will show you an animated progress bar
#Verbose=2 (one line per epoch)
opcion_muestra=1
neuronas_salida=1

division_validacion=0.2 #porcentaje de entrenamiento que se deja para validacion
dimension=sensores.shape[1] #esto representa el numero de columnas del patron sensorial

x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
y_train=y_train.reshape(y_train.shape[0],y_train.shape[1],1)
print ("shape x_train ",x_train.shape,"shape y_train ",y_train.shape)


#Ahora construimos el modelo de red neuronal
input_shape=keras.layers.Input(shape=(x_train.shape[1],1))
tower_1=keras.layers.MaxPool1D(5,strides=1)(input_shape)

tower_2=keras.layers.MaxPool1D(3)(input_shape)

tower_3=keras.layers.AveragePooling1D(pool_size=5,strides=1)(input_shape)
capas_paralelas=keras.layers.concatenate([tower_1,tower_2,tower_3], axis=1)
parte_primera=tf.keras.Model(inputs=input_shape, outputs=capas_paralelas)


modelo=tf.keras.models.Sequential()
modelo.add(parte_primera)


modelo.add(keras.layers.Flatten())

modelo.add(keras.layers.Dense(capa_oculta_1,name='capa_1',activation='tanh'))
modelo.add(keras.layers.Dense(capa_oculta_2,name='capa_2',activation='tanh'))
modelo.add(keras.layers.Dense(capa_oculta_3,name='capa_3',activation='relu',kernel_regularizer=l2(0.01)))
modelo.add(keras.layers.Dense(neuronas_salida,name='capa_4',activation='relu',kernel_regularizer=l2(0.01)))
#resumen del modelo
modelo.summary()
input ("ha llegado el momento de entrenar ... ")

modelo.compile(optimizer='adam',loss='mean_squared_error', metrics=['mean_squared_error'])
#entrenamos el modelo
modelo.fit(x_train,y_train,batch_size=tamanho_batch,epochs=epocas,verbose=opcion_muestra,validation_split=division_validacion)
#evaluamos  el modelo

entrenamiento_loss,entrenamiento_acc=modelo.evaluate(x_train,y_train)
print ("resultado sobre entrenamiento  ",entrenamiento_acc, " loss ",entrenamiento_loss)

x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)
y_test=y_test.reshape(y_test.shape[0],y_test.shape[1],1)
print ("shape x_train ",x_test.shape,"shape y_train ",y_test.shape)

test_loss,test_acc=modelo.evaluate(x_test,y_test)
print ("resultado sobre test ",test_acc," loss ",test_loss)


salida_test=modelo(x_test)
print (salida_test.shape)
input ("pulsa para verificar")

for i in range (salida_test.shape[0]):
    print ("salida red ",(float)(salida_test[i])," salida deseada ",y_test[i])
    if (i % 100 == 0):
        input ("pulsa para seguir ....")



